In [1]:
# import internal files
from historymatch import emulators
from historymatch import sample
from historymatch import historymatch
from historymatch import plot
from historymatch import utils


# import external modules
#import matplotlib.patches as patches
#import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#from scipy import stats
import math
import os
import pickle
#from matplotlib.patches import Rectangle


plt.rcParams.update({'font.size': 10})

np.random.seed(4)

In [2]:
# import data

with open("data/MassEval2016.dat",'r') as infile:
    Masses = pd.read_fwf(infile, usecols=(2,3,4,6,11,12),
              names=('N', 'Z', 'A', 'Element', 'Ebinding', 'E_unc'),
              widths=(1,3,5,5,5,1,3,4,1,13,11,11,9,1,2,11,9,1,3,1,12,11,1),
              header=64,
              index_col=False)
    
# Extrapolated values are indicated by '#' in place of the decimal place, so
# the Ebinding column won't be numeric. Coerce to float and drop these entries.
Masses['Ebinding'] = pd.to_numeric(Masses['Ebinding'], errors='coerce')
Masses = Masses.dropna()
Masses['E_unc'] = pd.to_numeric(Masses['E_unc'], errors='coerce')
Masses = Masses.dropna()
# Convert from keV to MeV.
Masses['Ebinding'] /= 1000
Masses['E_unc'] /= 1000

# Find the rows of the grouped DataFrame with the maximum binding energy.
Masses_single = Masses.groupby('A').apply(lambda t: t[t.Ebinding==t.Ebinding.max()])

A0_single = Masses_single['A'].to_numpy()
Z0_single = Masses_single['Z'].to_numpy()
N0_single = Masses_single['N'].to_numpy()
Element_single = Masses_single['Element'].to_numpy()

# convert energies per nucleon to energies
Energies0_single = Masses_single['Ebinding'].to_numpy()

Energies_unc0_single = Masses_single['E_unc'].to_numpy()

# extract 3 nucleon around maximum
Masses_triple = Masses.sort_values(['A','Ebinding'],ascending=False).groupby('A').head(3)

A0_triple = Masses_triple['A'].to_numpy()
Z0_triple = Masses_triple['Z'].to_numpy()
N0_triple = Masses_triple['N'].to_numpy()
Element_triple = Masses_triple['Element'].to_numpy()

# convert energies per nucleon to energies
Energies0_triple = Masses_triple['Ebinding'].to_numpy()

Energies_unc0_triple = Masses_triple['E_unc'].to_numpy()

In [3]:
# define parameter space

theta_0_bound = np.array([0, 40]).reshape(1,-1)
theta_1_bound = np.array([-40, 10]).reshape(1,-1)
theta_2_bound = np.array([-3, 3]).reshape(1,-1)
theta_3_bound = np.array([-50, 0]).reshape(1,-1) # keep
theta_4_bound = np.array([0, 40]).reshape(1,-1)

parameter_bounds = np.concatenate((theta_0_bound, theta_1_bound, \
                                   theta_2_bound, theta_3_bound, theta_4_bound), axis=0)


In [4]:
def LiquidDropModel(params, A, Z, N):
    
    if len(params) == 4:
        a1, a2, a3, a4 = params
        EB = (a1*A + a2*(A**(2.0/3.0)) + a3*Z*(Z-1)*(A**(-1.0/3.0)) \
            + a4*((N-Z)**2)/A)
        return EB/A
    else:
        a1, a2, a3, a4, a5 = params
        EB = (a1*A + a2*(A**(2.0/3.0)) + a3*Z*(Z-1)*(A**(-1.0/3.0)) \
            + a4*((N-Z)**2)/A + a5*( (-1)**Z + (-1)**N )*(2*A**(-0.5)))
        return EB/A
    

In [5]:
def create_model_error(_A):
    
    sigma_model = np.zeros(len(_A))     
        
    for i in range(len(_A)):
        if _A[i] < 40:
            sigma_model[i] = 0.09


        elif _A[i] < 140:
            sigma_model[i] = 0.05

        elif _A[i] < 200:
            sigma_model[i] = 0.04
        else:
            sigma_model[i] = 0.02
            
    return sigma_model


In [6]:
# define observational data for each wave

# wave 1 - A < 40
obs_data_wave1 = Energies0_single[10:30:2]
A_wave1 = A0_single[10:30:2]
Z_wave1 = Z0_single[10:30:2]
N_wave1 = N0_single[10:30:2]
variables_wave1 = np.concatenate((A_wave1.reshape(-1,1),Z_wave1.reshape(-1,1),\
                            N_wave1.reshape(-1,1)), axis=1)

sigma_obs_wave1 = Energies_unc0_single[10:30:2]
sigma_model_wave1 = create_model_error(A_wave1)


# wave 2 - A < 140
obs_data_wave2 = Energies0_single[10:130:3]
A_wave2 = A0_single[10:130:3]
Z_wave2 = Z0_single[10:130:3]
N_wave2 = N0_single[10:130:3]
variables_wave2 = np.concatenate((A_wave2.reshape(-1,1),Z_wave2.reshape(-1,1),\
                            N_wave2.reshape(-1,1)), axis=1)

sigma_obs_wave2 = Energies_unc0_single[10:130:3]
sigma_model_wave2 = create_model_error(A_wave2)


# wave 3 - A < 200
obs_data_wave3 = Energies0_single[10:190:3]
A_wave3 = A0_single[10:190:3]
Z_wave3 = Z0_single[10:190:3]
N_wave3 = N0_single[10:190:3]
variables_wave3 = np.concatenate((A_wave3.reshape(-1,1),Z_wave3.reshape(-1,1),\
                            N_wave3.reshape(-1,1)), axis=1)

sigma_obs_wave3 = Energies_unc0_single[10:190:3]
sigma_model_wave3 = create_model_error(A_wave3)


# wave 4 - A > 200
obs_data_wave4 = Energies0_single[10:-1:3]
A_wave4 = A0_single[10:-1:3]
Z_wave4 = Z0_single[10:-1:3]
N_wave4 = N0_single[10:-1:3]
variables_wave4 = np.concatenate((A_wave4.reshape(-1,1),Z_wave4.reshape(-1,1),\
                            N_wave4.reshape(-1,1)), axis=1)

sigma_obs_wave4 = Energies_unc0_single[10:-1:3]
sigma_model_wave4 = create_model_error(A_wave4)


# wave 5 - introduce ap - all data such that 3 points around E minimum are captured
obs_data_wave5 = np.flip(Energies0_triple)[30:-1:4]
A_wave5 = np.flip(A0_triple)[30:-1:4]
Z_wave5 = np.flip(Z0_triple)[30:-1:4]
N_wave5 = np.flip(N0_triple)[30:-1:4]
variables_wave5 = np.concatenate((A_wave5.reshape(-1,1),Z_wave5.reshape(-1,1),\
                            N_wave5.reshape(-1,1)), axis=1)

sigma_obs_wave5 = np.flip(Energies_unc0_triple)[30:-1:4]
sigma_model_wave5 = create_model_error(A_wave5)


obs_data = [obs_data_wave1, obs_data_wave2, obs_data_wave3, obs_data_wave4, obs_data_wave5, obs_data_wave5]
sigma_obs = [sigma_obs_wave1, sigma_obs_wave2, sigma_obs_wave3, sigma_obs_wave4, sigma_obs_wave5, sigma_obs_wave5]
sigma_model = [sigma_model_wave1, sigma_model_wave2, sigma_model_wave3, sigma_model_wave4, sigma_model_wave5, sigma_model_wave5]

variables = [variables_wave1, variables_wave2, variables_wave3, variables_wave4, variables_wave5, variables_wave5]

sigma_method = [np.zeros_like(sigmas) for sigmas in sigma_model]

# History Matching - Separate Waves

In [7]:
volshapes = ['gaussian', 'hypercube', 'hypercube_rot', 'ellipsoid']

In [8]:
nwaves = 5
ndim = 5
nsamples = 1 * (10**6)
volshape = 'ellipsoid'

In [9]:
# initialise history matching class
HM = historymatch.HistoryMatch(ndim, filename='result_dict', emulator='GP', volume_shape=volshape)
# initialise results class
Results = historymatch.Results('resultfile')

In [10]:


ToyModel = historymatch.Simulator(HM)
ToyModel.set_simulator(LiquidDropModel)

HM.initialize_volume(parameter_bounds[:,0], parameter_bounds[:,1], ninactive=1, inactive_wave = 5, sigma_inactive=0.02)


#wave1 = HM.run_wave(1, obs_data[0], sigma_obs[0], sigma_model[0], sigma_method[0], variables[0], nsamples=nsamples, ntraining=100)

In [11]:
#HM.store_result(Results, wave=1, wave_results = wave1)

In [12]:
#results_wave2 = HM.run_wave(2, obs_data[1], sigma_obs[1], sigma_model[1], sigma_method[1], variables[1], nsamples=nsamples)

In [13]:
#results_wave2

In [14]:
#HM.store_result(Results, wave=2, wave_results = results_wave2)

In [15]:
# plot wave 2 results
#plot.plotcorner(Results.samples_I[1], parameter_bounds, ndim-1)

# History Match - All Waves

In [16]:
# initialise all observational data and run multiple waves
HM.set_observations(obs_data, variables=variables, sigma_obs=sigma_obs, sigma_model=sigma_model, sigma_method=sigma_method)

results = HM.run(nwaves=nwaves, ntraining=100, nsamples=nsamples)

Running wave 1


100%|██████████| 10/10 [00:38<00:00,  3.84s/it]


Number of Non-Implausible Samples: 659
Running wave 2
ellipsoid


100%|██████████| 40/40 [02:36<00:00,  3.91s/it]


Number of Non-Implausible Samples: 5864
Running wave 3
ellipsoid


100%|██████████| 60/60 [03:50<00:00,  3.85s/it]


Number of Non-Implausible Samples: 158376
Running wave 4
ellipsoid


 36%|███▌      | 30/83 [01:54<03:21,  3.80s/it]


KeyboardInterrupt: 

In [ ]:
# initialise history matching class
HM2 = historymatch.HistoryMatch(ndim, filename='result_dict', emulator='GP', volume_shape='hypercube_rot')
# initialise results class
Results2 = historymatch.Results('resultfile')

ToyModel2 = historymatch.Simulator(HM2)
ToyModel2.set_simulator(LiquidDropModel)

HM2.initialize_volume(parameter_bounds[:,0], parameter_bounds[:,1], ninactive=1, inactive_wave = 5, sigma_inactive=0.02)

HM2.set_observations(obs_data, variables=variables, sigma_obs=sigma_obs, sigma_model=sigma_model, sigma_method=sigma_method)

results_hc_rot = HM2.run(nwaves=nwaves, ntraining=100, nsamples=nsamples)

In [ ]:
fig, ax = plt.subplots(figsize=(5,10))

ax.scatter(results_hc_rot.nonimplausible[0][:,0],results_hc_rot.nonimplausible[0][:,1], alpha=0.1, zorder=5, color='black')

ax.scatter(results.samples_I[1][:,0],results.samples_I[1][:,1])
ax.scatter(results_hc_rot.samples_I[1][:,0],results_hc_rot.samples_I[1][:,1])
plt.axis('square')

ax.set_xlim([0,30])

In [ ]:
fig, ax = plt.subplots(figsize=(5,10))

ax.scatter(results_hc_rot.nonimplausible[0][:,0],results_hc_rot.nonimplausible[0][:,1], alpha=0.1, zorder=5, color='black')
ax.scatter(results.nonimplausible[0][:,0],results.nonimplausible[0][:,1], alpha=0.1, zorder=5, color='red')

#ax.scatter(results.samples_I[1][:,0],results.samples_I[1][:,1])
#ax.scatter(results_hc_rot.samples_I[1][:,0],results_hc_rot.samples_I[1][:,1])
plt.axis('square')

ax.set_xlim([0,30])

In [ ]:
testcov = np.array([[3,2],[2,3]])
_eigvals, eigvecs = np.linalg.eig(testcov)

testpts = np.random.multivariate_normal([0,0], testcov,1000)
R = eigvecs.T
testpts2 = np.dot(testpts,R)
plt.scatter(testpts[:,0], testpts[:,1])
#plt.scatter(testpts2[:,0], testpts2[:,1])
#plt.scatter(0,0)
print(2*2*np.sqrt(_eigvals))


In [ ]:
covariance = np.cov(results_hc_rot.nonimplausible[0][:,:-1].T)

_eigvals, eigvecs = np.linalg.eig(covariance)
print(covariance)
R = eigvecs.T                            # rotation matrix
S = 2*np.eye(4)*np.sqrt(_eigvals)     # scaling matrix
T = np.dot(S, np.linalg.inv(R))          # tranformation matrix

print(S)
scaled_samples = np.zeros((len(results_hc_rot.samples_I[1]),4))
for i in range(len(results_hc_rot.samples_I[1])):
    scaled_samples[i] = np.dot(R, results_hc_rot.samples_I[1][i,:4])
    
    
fig, ax = plt.subplots(figsize=(5,10))

ax.scatter(results_hc_rot.samples_I[1][:,0],results_hc_rot.samples_I[1][:,1])
#ax.scatter(scaled_samples[:,0],scaled_samples[:,1])
plt.axis('square')

#ax.set_xlim([0,30])

In [ ]:
for k in range(5):
    
    
    # number of nonimplausible samples
    print(hypercube_rot_nonimplausible[k].shape)
    
    # rotate to straight
    covariance = np.cov(hypercube_rot_samples[k+1][:,:-1].T)
    _eigvals, eigvecs = np.linalg.eigh(covariance)

    R = eigvecs.T
    nonimp_straight = hypercube_rot_samples[k+1][:,:-1].dot(np.linalg.inv(R))
    if k < 4:
        hc_rot_bounds = utils.locate_boundaries(nonimp_straight, 4)
        hc_rot_volume = 1
        initial_volume = 1
        for i in range(4):
            initial_volume *= np.abs(initial_parameter_bounds[i,1]-initial_parameter_bounds[i,0])
            hc_rot_volume *= np.abs(hc_rot_bounds[i,1]-hc_rot_bounds[i,0]) 
    else:
        hc_rot_bounds = utils.locate_boundaries(nonimp_straight, 5)
        hc_rot_volume = 1
        initial_volume = 1
        for i in range(5):
            initial_volume *= np.abs(initial_parameter_bounds[i,1]-initial_parameter_bounds[i,0])
            hc_rot_volume *= np.abs(hc_rot_bounds[i,1]-hc_rot_bounds[i,0])
    print(hc_rot_volume)
    
    print('Rotated Hypercube volume  : ' + str(100*np.prod(4*np.sqrt(_eigvals))/initial_volume) + '%')